In [1]:
import praw as praw
import os
from dotenv import load_dotenv
import re
import transformers
from transformers import pipeline
import pandas as pd
load_dotenv()

C:\Users\dfaro\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
C:\Users\dfaro\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


True

In [2]:
# authenticating with reddit and setting up an object for interacting with the API

reddit = praw.Reddit(
    client_id=os.environ["reddit_client_id"],
    client_secret=os.environ["reddit_client_secret"],
    user_agent=os.environ["reddit_user_agent"],
)


In [3]:
# Create a regular expression pattern to match URLs
url_pattern = re.compile(r'https?://[^\s]+')

In [4]:
def grab_posts(subreddit,num_posts):

    # grab "x" number of posts in a subreddit and store them in a variable
    # the reasoning is because if you loop in reddit.subreddit(subreddit).hot(limit=num_posts), it returns a generator object, and the generator is being exhausted after the first iteration. This would result in empty submission text after the first iterration
    # It WILL SKIP posts with ONLY IMAGES
    posts = list(reddit.subreddit(subreddit).hot(limit=num_posts))

     # dataframe
    data = {'submission_title': [],'submission_post': [], 'upvotes': [], 'downvotes': []}
    # data = {'submission(post)': [], 'upvotes': [], 'downvotes': [], 'scoring': []}
    df = pd.DataFrame(data)

    for submission in posts:
        # reset variables
        submission_title = submission.title
        submission_text = submission.selftext
        upvotes = 0
        downvotes = 0

        # Use the sub method to remove all URLs from the submission text
        stripped_submission = url_pattern.sub('', submission_text)

        # if stripped_submission != "":
        print('this is the submission text: ',stripped_submission)
        

        # total upvotes for submission
        upvotes = int(submission.ups)
        print(upvotes)

        # total downvotes for submission
        downvotes = int(submission.downs)
        print(downvotes)

        # # loop through each comment and grab the text, upvotes, and downvotes
        # for comment in submission.comments.top(limit = 1):
        #     # Use the sub method to remove all URLs from the comment body
        #     stripped_comment = url_pattern.sub('', comment.body)
        #     print(stripped_comment)s
        #     print(comment.ups)
        #     print(comment.downs)

        test = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")
        output = test(submission_text)
        print(output)
        for item in output:
            print(item['score'])
        # # Append the data to the dataframe
        # df = df.append({'submission(post)': stripped_submission, 'upvotes': upvotes, 'downvotes': downvotes, 'scoring':scoring}, ignore_index=True)
        df = df.append({'submission_title': submission_title, 'submission_post': stripped_submission, 'upvotes': upvotes, 'downvotes': downvotes}, ignore_index=True)

        # Write the dataframe to a CSV file
        # This will create a file called posts.csv in the current working directory, and store the submissions(post), their upvotes and downvotes in it.
        df.to_csv('posts.csv', index=False)

In [5]:
def main():
    # Get subreddit from user input
    subreddit = input("Enter the subreddit name: ")

    # Get number of hottest posts to look at from user input
    num_posts = int(input("Enter the number of subreddit posts to look at: "))

    grab_posts(subreddit,num_posts)

In [7]:
df1 = pd.read_csv('ultimatelist.csv',header = None)

In [8]:
to_drop = df1.loc[df1[2]=='nsfw=true']

In [9]:
to_drop

,0,1,2
62,1424738,gonewild,nsfw=true
86,932952,nsfw,nsfw=true
95,899550,NSFW_GIF,nsfw=true
101,873355,RealGirls,nsfw=true
121,771699,ImGoingToHellForThis,nsfw=true
...,...,...,...
1088518,0,0u812,nsfw=true
1088527,0,0Like,nsfw=true
1088608,0,010199,nsfw=true
1088612,0,0_0Wut,nsfw=true


In [10]:
df1

,0,1,2
0,24209578,announcements,nsfw=false
1,19406878,funny,nsfw=false
2,19129743,AskReddit,nsfw=false
3,18724463,todayilearned,nsfw=false
4,18628755,worldnews,nsfw=false
...,...,...,...
1088634,0,000002,nsfw=false
1088635,0,00000000control,nsfw=false
1088636,0,0000000027544444,nsfw=false
1088637,-1,ramblin,nsfw=false


In [11]:
df2 = df1.loc[df1[2]=='nsfw=false']

In [12]:
df2

,0,1,2
0,24209578,announcements,nsfw=false
1,19406878,funny,nsfw=false
2,19129743,AskReddit,nsfw=false
3,18724463,todayilearned,nsfw=false
4,18628755,worldnews,nsfw=false
...,...,...,...
1088633,0,000003,nsfw=false
1088634,0,000002,nsfw=false
1088635,0,00000000control,nsfw=false
1088636,0,0000000027544444,nsfw=false


In [13]:
df2 = df2.reset_index()

In [14]:
df2

,index,0,1,2
0,0,24209578,announcements,nsfw=false
1,1,19406878,funny,nsfw=false
2,2,19129743,AskReddit,nsfw=false
3,3,18724463,todayilearned,nsfw=false
4,4,18628755,worldnews,nsfw=false
...,...,...,...,...
1024280,1088633,0,000003,nsfw=false
1024281,1088634,0,000002,nsfw=false
1024282,1088635,0,00000000control,nsfw=false
1024283,1088636,0,0000000027544444,nsfw=false


In [15]:
df3 = df2[:100]

In [16]:
df3

,index,0,1,2
0,0,24209578,announcements,nsfw=false
1,1,19406878,funny,nsfw=false
2,2,19129743,AskReddit,nsfw=false
3,3,18724463,todayilearned,nsfw=false
4,4,18628755,worldnews,nsfw=false
...,...,...,...,...
95,98,880981,buildapc,nsfw=false
96,99,880748,FoodPorn,nsfw=false
97,100,879513,instant_regret,nsfw=false
98,102,868126,GifRecipes,nsfw=false


In [17]:
df3.to_csv('Top100.csv')

In [ ]:
df3 = df3.drop('index', axis = 1)

In [ ]:
df3

In [ ]:
df3 = df3.rename(columns={0:'subscriber_count', 1:'subreddit'})

In [ ]:
df3 = df3.drop(2,axis = 1)

In [ ]:
df3.to_csv('Top5000Subreddits.csv')

In [ ]:
df4 = pd.read_csv('Top5000Subreddits.csv')

In [ ]:
df4

In [ ]:
df4 = df4.drop('subreddi1',axis=1)

In [ ]:
df4

In [ ]:
df4 = df4.dropna()

In [ ]:
df4

In [ ]:
df4.to_csv('Top5000_Subreddits.csv')

In [ ]:
df4.loc[df4['subreddit']=='instant_regret']

In [ ]:
pip install -r anaconda-environment.txt

In [ ]:
import json

output = "[[{'label': 'NEG', 'score': 0.9633743166923523}, {'label': 'NEU', 'score': 0.03375086933374405}, {'label': 'POS', 'score': 0.0028747699689120054}]]"

output_list = json.loads(output)
